In [1]:
import pandas as pd
import numpy as np

import nltk                            # инструменты для обработки естественного языка
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import re                              # библиотека для работы с регулярными выражениями
import string                          # модуль для работы со строками
from pymystem3 import Mystem           # инструмент для работы c русским текстом

import glob
import os
import time

from tqdm import tqdm                  # добавляет progress bar к циклам
from joblib import Parallel, delayed   # инструмент для параллельного программирования

Читаем все csv файлы с комментариями в папке

In [2]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path + "\women_ru", "*.csv"))

women_ru = []
for f in csv_files:
    women_ru.append(pd.read_csv(f))

women_ru = pd.concat(women_ru, ignore_index=True)
display(women_ru)


,etag,parentid,id,textDisplay,textOriginal,authorDisplayName,authorProfileImageUrl,authorChannelUrl,authorChannelId,likeCount,publishedAt,updatedAt
0,hguN93Z84IVQ_d1CGZiTLRyhV3g,Ugz0AvyC0raprzzGu3B4AaABAg,Ugz0AvyC0raprzzGu3B4AaABAg,Хотите работать в Кремниевой долине? Релокейт ...,Хотите работать в Кремниевой долине? Релокейт ...,Victoria Borodina,https://yt3.ggpht.com/ytc/AMLnZu9yfkx926mlkkTt...,http://www.youtube.com/channel/UC1QrUyRmSoUTPV...,UC1QrUyRmSoUTPVSbHv7arcA,9,2020-03-25T05:13:10Z,2020-03-25T05:13:10Z
1,il95efMvBOf3rq1uKayDaQx-GqA,UgyniiD0GA5HFUe_S7N4AaABAg,UgyniiD0GA5HFUe_S7N4AaABAg,2 года уже прошло,2 года уже прошло,Костя М,https://yt3.ggpht.com/ytc/AMLnZu8ZwBzjYEu_EUAw...,http://www.youtube.com/channel/UCy1mYHfrlaBoAi...,UCy1mYHfrlaBoAinHffRIAcQ,0,2022-05-10T14:06:37Z,2022-05-10T14:06:48Z
2,LoLBD02TInId7P2lE2QnQxuVh3w,UgxtFtTRB9iYB8i1ssd4AaABAg,UgxtFtTRB9iYB8i1ssd4AaABAg,"В IT исторически шли мужчины, как было сказано...","В IT исторически шли мужчины, как было сказано...",nBrains,https://yt3.ggpht.com/ytc/AMLnZu-HRPynRFVZwiy4...,http://www.youtube.com/channel/UCgNh88PuFxq7C_...,UCgNh88PuFxq7C_C5dKY-FZg,1,2022-01-19T11:13:29Z,2022-01-19T11:14:39Z
3,inr5XKQxUWVBFr2l_Ih68mdIZsw,Ugz2fkzgIBD8buAWyxN4AaABAg,Ugz2fkzgIBD8buAWyxN4AaABAg,Канал классный все збс. НО чтоб коммент мой не...,Канал классный все збс. НО чтоб коммент мой не...,Карпатські Гори,https://yt3.ggpht.com/ytc/AMLnZu85HxJpLgagbX8n...,http://www.youtube.com/channel/UCaGPuwtwb_ysIR...,UCaGPuwtwb_ysIR6DtonHzfw,0,2021-10-10T13:57:09Z,2021-10-10T13:57:09Z
4,VwwZjcgBuBH14HGoMHjMmtUolBA,UgxULso55foCfIQ99Kl4AaABAg,UgxULso55foCfIQ99Kl4AaABAg,"Когда первый раз посмотрел это видео, то подум...","Когда первый раз посмотрел это видео, то подум...",Carpe Diem,https://yt3.ggpht.com/ytc/AMLnZu8IEAQVW7T7hzAV...,http://www.youtube.com/channel/UCjjXODb0eImfDa...,UCjjXODb0eImfDa-l0tW67sA,8,2021-09-21T18:28:19Z,2021-09-21T18:28:19Z
...,...,...,...,...,...,...,...,...,...,...,...,...
3596,mD1xoFQbMOsdWFrmHxg_PoKjK7Q,UgzgH0JoOpCE3cTTcvp4AaABAg,UgzgH0JoOpCE3cTTcvp4AaABAg,С каждым подписчиком буду писать &quot;Спасибо...,"С каждым подписчиком буду писать ""Спасибо"", се...",study with xenia,https://yt3.ggpht.com/ytc/AMLnZu_BYx_arXxBf1He...,http://www.youtube.com/channel/UCrnFm0wUI_im18...,UCrnFm0wUI_im18_qOtqeO6A,7,2020-08-19T14:01:56Z,2020-08-19T14:02:13Z
3597,oTGWh7xyelrUGh7QFdUHPY_5n_0,Ugx--D0quwAwxPvJ6sF4AaABAg,Ugx--D0quwAwxPvJ6sF4AaABAg,Ждал этого. Все программисты ставят лайк.,Ждал этого. Все программисты ставят лайк.,MichDeve,https://yt3.ggpht.com/kh0DgQA1389OOIfpRgVKDkDG...,http://www.youtube.com/channel/UCGF9uIr1ZjrVUe...,UCGF9uIr1ZjrVUeQiw2S5HYw,747,2020-08-19T14:01:45Z,2020-08-19T14:01:45Z
3598,ZITDZCYrU-PvnffN__8eKGx5_kg,Ugy7BuGfKiLV2Cgd62R4AaABAg,Ugy7BuGfKiLV2Cgd62R4AaABAg,Пр,Пр,Aisana Arstanaliyeva,https://yt3.ggpht.com/ytc/AMLnZu9oPbOXGvwi8XNP...,http://www.youtube.com/channel/UC5Gw7CsQDLEsbu...,UC5Gw7CsQDLEsbuBVHBQD5Sg,0,2020-08-19T14:01:23Z,2020-08-19T14:01:23Z
3599,3CPI2tgVwdtJQ0xR9ihAkfedivw,Ugwh4GFX5Zwx7lNXUqp4AaABAg,Ugwh4GFX5Zwx7lNXUqp4AaABAg,первый пошёл,первый пошёл,кусок овчарки,https://yt3.ggpht.com/ytc/AMLnZu_HAGctptAQIur0...,http://www.youtube.com/channel/UC8886eDRUAxnph...,UC8886eDRUAxnphgFtTsD9Xw,1,2020-08-19T14:01:08Z,2020-08-19T14:01:08Z


In [3]:
women_ru = women_ru[['id', 'textOriginal', 'likeCount']]
display(women_ru)

,id,textOriginal,likeCount
0,Ugz0AvyC0raprzzGu3B4AaABAg,Хотите работать в Кремниевой долине? Релокейт ...,9
1,UgyniiD0GA5HFUe_S7N4AaABAg,2 года уже прошло,0
2,UgxtFtTRB9iYB8i1ssd4AaABAg,"В IT исторически шли мужчины, как было сказано...",1
3,Ugz2fkzgIBD8buAWyxN4AaABAg,Канал классный все збс. НО чтоб коммент мой не...,0
4,UgxULso55foCfIQ99Kl4AaABAg,"Когда первый раз посмотрел это видео, то подум...",8
...,...,...,...
3596,UgzgH0JoOpCE3cTTcvp4AaABAg,"С каждым подписчиком буду писать ""Спасибо"", се...",7
3597,Ugx--D0quwAwxPvJ6sF4AaABAg,Ждал этого. Все программисты ставят лайк.,747
3598,Ugy7BuGfKiLV2Cgd62R4AaABAg,Пр,0
3599,Ugwh4GFX5Zwx7lNXUqp4AaABAg,первый пошёл,1


Тестовая строка

In [4]:
test = women_ru.iloc[2, 1]
print(test)
print('======================')

test_clean = re.sub(r'[^\w\s]', '', test)
print(test_clean)
print('======================')

m = Mystem()
test_lemmas = m.lemmatize(test_clean)
print(test_lemmas)
print('======================')

test_lemmas = [l for l in test_lemmas if (l != '\n' and l.strip() != '')]
print(test_lemmas)


В IT исторически шли мужчины, как было сказано в видео, в корне не верно, наоборот первыми программистами как раз были женщины, в целом эта профессия считалась женской. Первый компилятор и буквально сам термин компилировать придумала женщина.
В IT исторически шли мужчины как было сказано в видео в корне не верно наоборот первыми программистами как раз были женщины в целом эта профессия считалась женской Первый компилятор и буквально сам термин компилировать придумала женщина
['в', ' ', 'IT', ' ', 'исторически', ' ', 'идти', ' ', 'мужчина', ' ', 'как', ' ', 'быть', ' ', 'сказать', ' ', 'в', ' ', 'видео', ' ', 'в', ' ', 'корень', ' ', 'не', ' ', 'верный', ' ', 'наоборот', ' ', 'первый', ' ', 'программист', ' ', 'как', ' ', 'раз', ' ', 'быть', ' ', 'женщина', ' ', 'в', ' ', 'целое', ' ', 'этот', ' ', 'профессия', ' ', 'считаться', ' ', 'женский', ' ', 'первый', ' ', 'компилятор', ' ', 'и', ' ', 'буквально', ' ', 'сам', ' ', 'термин', ' ', 'компилировать', ' ', 'придумывать', ' ', 'женщина

Удаляем стоп-слова, если это нужно

In [5]:
#nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
print(stop_words)
print('======================')

test_non_stop = [l for l in test_lemmas if l not in stop_words]
print(' '.join(test_non_stop))

{'не', 'куда', 'чтоб', 'этот', 'больше', 'им', 'от', 'него', 'я', 'зачем', 'он', 'тот', 'ну', 'том', 'чем', 'ему', 'мой', 'вдруг', 'почти', 'там', 'три', 'такой', 'были', 'них', 'тут', 'когда', 'вот', 'может', 'два', 'свою', 'никогда', 'тем', 'тогда', 'мы', 'этого', 'о', 'под', 'так', 'разве', 'что', 'между', 'тебя', 'но', 'было', 'если', 'вас', 'моя', 'над', 'про', 'вам', 'со', 'вы', 'ты', 'да', 'впрочем', 'в', 'и', 'она', 'даже', 'на', 'можно', 'у', 'ведь', 'всю', 'мне', 'всех', 'ничего', 'нее', 'бы', 'уже', 'здесь', 'был', 'еще', 'более', 'ее', 'без', 'надо', 'потом', 'чуть', 'того', 'или', 'этой', 'перед', 'ж', 'ним', 'через', 'себе', 'себя', 'как', 'с', 'где', 'лучше', 'а', 'совсем', 'кто', 'ни', 'они', 'ей', 'хоть', 'во', 'эти', 'их', 'за', 'меня', 'из', 'будет', 'чтобы', 'быть', 'уж', 'ли', 'много', 'до', 'потому', 'об', 'наконец', 'после', 'же', 'чего', 'его', 'то', 'по', 'будто', 'один', 'конечно', 'хорошо', 'для', 'все', 'только', 'нельзя', 'иногда', 'нет', 'раз', 'нас', 'ней

Небольшой текст можно обработать с помощью следующей функции. Однако на корпусе она будет работать крайне медленно

In [6]:
#m = Mystem()
#stop_words = set(stopwords.words('russian'))
def clean_text(text):
    start_time = time.time()
    
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    cleaned_text = m.lemmatize(cleaned_text)
    cleaned_text = [l for l in test_lemmas if (l != '\n' and l.strip() != '')]
    cleaned_text = [l for l in test_lemmas if l not in stop_words]
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return ' '.join(cleaned_text)

print(clean_text(test))

--- 1.0753440856933594 seconds ---
IT исторически идти мужчина сказать видео корень верный наоборот первый программист женщина целое профессия считаться женский первый компилятор буквально термин компилировать придумывать женщина


Чистим тексты от пунктуации

In [8]:
women_ru['clean_text'] = women_ru['textOriginal'].map(lambda text: re.sub(r'[^\w\s]', '', text))
display(women_ru)

,id,textOriginal,likeCount,clean_text
0,Ugz0AvyC0raprzzGu3B4AaABAg,Хотите работать в Кремниевой долине? Релокейт ...,9,Хотите работать в Кремниевой долине Релокейт в...
1,UgyniiD0GA5HFUe_S7N4AaABAg,2 года уже прошло,0,2 года уже прошло
2,UgxtFtTRB9iYB8i1ssd4AaABAg,"В IT исторически шли мужчины, как было сказано...",1,В IT исторически шли мужчины как было сказано ...
3,Ugz2fkzgIBD8buAWyxN4AaABAg,Канал классный все збс. НО чтоб коммент мой не...,0,Канал классный все збс НО чтоб коммент мой не ...
4,UgxULso55foCfIQ99Kl4AaABAg,"Когда первый раз посмотрел это видео, то подум...",8,Когда первый раз посмотрел это видео то подума...
...,...,...,...,...
3596,UgzgH0JoOpCE3cTTcvp4AaABAg,"С каждым подписчиком буду писать ""Спасибо"", се...",7,С каждым подписчиком буду писать Спасибо сейча...
3597,Ugx--D0quwAwxPvJ6sF4AaABAg,Ждал этого. Все программисты ставят лайк.,747,Ждал этого Все программисты ставят лайк
3598,Ugy7BuGfKiLV2Cgd62R4AaABAg,Пр,0,Пр
3599,Ugwh4GFX5Zwx7lNXUqp4AaABAg,первый пошёл,1,первый пошёл


Объединяем комментарии в большие фрагменты, обрабатываем с параллельным выполнением лемматизации

In [9]:
texts = women_ru['clean_text']
print(texts[0:3])

0    Хотите работать в Кремниевой долине Релокейт в...
1                                    2 года уже прошло
2    В IT исторически шли мужчины как было сказано ...
Name: clean_text, dtype: object


In [10]:
batch_size = 500
text_batch = [texts[i: i + batch_size] for i in range(0, len(texts), batch_size)]

def lemmatize(text):
    
    m = Mystem()
    merged_text = "|".join(text)

    doc = []
    res = []

    for t in m.lemmatize(merged_text):
        if t != '|':
            doc.append(t)
        else:
            res.append(doc)
            doc = []

    return res

processed_texts = Parallel(n_jobs=-1)(delayed(lemmatize)(t) for t in text_batch)

In [25]:
print(processed_texts[6])

[['давать', ' ', 'что', ' ', 'бесить', ' ', 'волонтер', ' ', 'или', ' ', 'доставщик', ' ', 'пицца'], ['ой', ' ', 'жиз', ' ', 'пояснювати', ' ', 'человек', ' ', 'чим', ' ', 'ти', ' ', 'займаєшся', ' ', 'тот', ' ', 'главный', ' ', 'що', ' ', 'не', ' ', 'простітуція', ' ', 'бо', ' ', 'тоді', ' ', 'мамі', ' ', 'ж', ' ', 'не', ' ', 'пояснять', ' |', 'у', ' ', 'вы', ' ', 'шедевральный', ' ', 'реклама'], ['как', ' ', 'же', ' ', 'приятно', ' ', 'наблюдать', ' ', 'за', ' ', 'этот', ' ', 'жестикуляция'], ['деньги', ' ', 'отличный', ' ', 'мотиватор', ' ', 'почему', ' ', 'все', ' ', 'твердить', ' ', 'обратный', ' ', 'человек', ' ', 'за', ' ', 'деньги', ' ', 'дерьмо', ' ', 'откачивать', ' ', 'траншея', ' ', 'копать', ' ', 'и', ' ', 'покойник', ' ', 'мыть', ' ', 'неужели', ' ', 'программист', ' ', 'это', ' ', 'нечто', ' ', 'возвышенный', ' ', 'где', ' ', 'деньги', ' ', 'не', ' ', 'мочь', ' ', 'быть', ' ', 'решающий', ' ', 'фактор', ' ', 'при', ' ', 'выбор', ' ', 'профессия', '\n', 'часто', ' ', 'вид

In [24]:
# не все комментарии разделились...

lemmas = [t for sublist in processed_texts for t in sublist]

print(len(processed_texts[6]), len(text_batch[6]))
print(' '.join([t for sublist in processed_texts[6] for t in sublist]), text_batch[6])



424 500
давать   что   бесить   волонтер   или   доставщик   пицца ой   жиз   пояснювати   человек   чим   ти   займаєшся   тот   главный   що   не   простітуція   бо   тоді   мамі   ж   не   пояснять  | у   вы   шедевральный   реклама как   же   приятно   наблюдать   за   этот   жестикуляция деньги   отличный   мотиватор   почему   все   твердить   обратный   человек   за   деньги   дерьмо   откачивать   траншея   копать   и   покойник   мыть   неужели   программист   это   нечто   возвышенный   где   деньги   не   мочь   быть   решающий   фактор   при   выбор   профессия 
 часто   видеть   человек   который   работать   на   стройка   и   они   говорить   это   мой   ради   деньги   я   бы   тут   не   работать   или   уборщик   официант   и   тд   ну   бред 
 кто   деньги   не   мотив   тот   пизливать   учить   програмирование   ради   бабнуть   деньги   отлично   мотивировать я   в   свой   глаз   включать   и   выключать   вайфай   чтобы   он   зарабатывать 
 я   в   глаз   бабуш